|       | Description |
| ----------- | ----------- |
| Author      | Honglin Qian (MSFE student at UIUC)|
| Date   | 02/12/2024        |
|Email|honglin6@illinois.edu|
|Idea| We price both American and European options using the fitted implied volatility from SSVI |

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as si
from scipy.stats import norm
import warnings
warnings.filterwarnings("ignore")
import QuantLib as ql
import plotly.graph_objects as go
import os
import regex as re
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from scipy import interpolate
from scipy.optimize import minimize
import seaborn as sns
from tqdm import tqdm
from datetime import datetime, timedelta

In [2]:
data_path = "./putcall.csv"
tmp = pd.read_csv(data_path, parse_dates=['QUOTE_DATE', 'EXPIRE_DATE'])
tmp

,QUOTE_DATE,EXPIRE_DATE,DTE,UNDERLYING_LAST,STRIKE,P_BID,P_ASK,P_DELTA,P_GAMMA,P_VEGA,...,C_GAMMA,C_VEGA,C_THETA,C_RHO,C_IV,P_price,C_price,rf,Log-Strike,div
0,2011-01-04,2011-01-07,3.00,1270.37,1050.0,0.00,0.05,-0.00146,0.00000,0.00498,...,0.00000,0.00000,0.00000,0.00000,NaN,0.025,219.450,0.001200,-0.190528,0.0183
1,2011-01-04,2011-01-07,3.00,1270.37,1075.0,0.00,0.05,-0.00157,0.00006,0.00550,...,0.00000,0.00000,0.00000,0.00000,NaN,0.025,194.450,0.001200,-0.166997,0.0183
2,2011-01-04,2011-01-07,3.00,1270.37,1100.0,0.00,0.05,-0.00114,0.00010,0.00635,...,0.00000,0.00000,0.00000,0.00000,NaN,0.025,169.450,0.001200,-0.144008,0.0183
3,2011-01-04,2011-01-07,3.00,1270.37,1125.0,0.00,0.05,-0.00119,0.00015,0.00654,...,0.00000,0.00000,0.00000,0.00000,NaN,0.025,144.455,0.001200,-0.121535,0.0183
4,2011-01-04,2011-01-07,3.00,1270.37,1150.0,0.05,0.15,-0.00626,0.00035,0.02337,...,0.00000,0.00000,0.00000,0.00000,NaN,0.100,119.450,0.001200,-0.099556,0.0183
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14704895,2023-09-29,2028-12-15,1904.04,4286.60,8800.0,2880.70,3072.60,-1.00000,0.00000,0.00000,...,0.00009,10.87378,-0.05829,11.09132,0.13139,2976.650,27.350,0.045752,0.480588,0.0156
14704896,2023-09-29,2028-12-15,1904.04,4286.60,9200.0,3186.30,3378.20,-1.00000,0.00000,0.00000,...,0.00007,9.24711,-0.04949,9.04105,0.13457,3282.250,22.050,0.045752,0.525040,0.0156
14704897,2023-09-29,2028-12-15,1904.04,4286.60,9600.0,3494.70,3686.60,-1.00000,0.00000,0.00000,...,0.00006,8.12839,-0.04411,7.69404,0.13909,3590.650,18.850,0.045752,0.567600,0.0156
14704898,2023-09-29,2028-12-15,1904.04,4286.60,10000.0,3805.00,3996.90,-1.00000,0.00000,0.00000,...,0.00000,6.16192,-0.03236,5.52154,0.13708,3900.950,12.700,0.045752,0.608422,0.0156


In [3]:
data_path = "./putcall.csv"

tmp = pd.read_csv(data_path, parse_dates=['QUOTE_DATE', 'EXPIRE_DATE'])

# 确保 QUOTE_DATE 列为 datetime64 类型
tmp['QUOTE_DATE'] = pd.to_datetime(tmp['QUOTE_DATE'])

# 筛选 2023 年的数据
tmp = tmp[tmp['QUOTE_DATE'].dt.year == 2023]

tmp = tmp[tmp['DTE'] == 365 * 2].reset_index(drop = True)

tmp

,QUOTE_DATE,EXPIRE_DATE,DTE,UNDERLYING_LAST,STRIKE,P_BID,P_ASK,P_DELTA,P_GAMMA,P_VEGA,...,C_GAMMA,C_VEGA,C_THETA,C_RHO,C_IV,P_price,C_price,rf,Log-Strike,div
0,2023-06-21,2025-06-20,730.0,4365.45,800.0,2.85,3.7,-0.00231,0.00000,0.51766,...,0.00000,0.00000,0.00000,0.00000,NaN,3.275,3538.050,0.046639,-1.790143,0.0165
1,2023-06-21,2025-06-20,730.0,4365.45,1000.0,4.60,5.0,-0.00377,0.00000,0.76942,...,0.00000,0.00000,0.00000,0.00000,NaN,4.800,3357.700,0.046639,-1.567000,0.0165
2,2023-06-21,2025-06-20,730.0,4365.45,1200.0,6.10,6.9,-0.00599,0.00004,1.06254,...,0.00005,1.50895,-0.04814,14.35172,0.45604,6.500,3177.650,0.046639,-1.384678,0.0165
3,2023-06-21,2025-06-20,730.0,4365.45,1400.0,8.00,9.0,-0.00821,0.00005,1.41332,...,-0.00001,2.26359,-0.08325,19.49766,0.44563,8.500,2998.000,0.046639,-1.230527,0.0165
4,2023-06-21,2025-06-20,730.0,4365.45,1600.0,10.60,11.5,-0.01145,-0.00004,1.84888,...,0.00006,3.00045,-0.11767,23.96140,0.42819,11.050,2820.650,0.046639,-1.096996,0.0165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,2023-06-21,2025-06-20,730.0,4365.45,6800.0,1910.50,1939.1,-1.00000,0.00000,0.00000,...,0.00011,3.55004,-0.03964,2.00062,0.12762,1924.800,6.800,0.046639,0.349923,0.0165
72,2023-06-21,2025-06-20,730.0,4365.45,7000.0,2093.60,2118.8,-1.00000,0.00000,0.00000,...,0.00008,2.72843,-0.03027,1.46588,0.13026,2106.200,4.850,0.046639,0.378911,0.0165
73,2023-06-21,2025-06-20,730.0,4365.45,7200.0,2270.90,2299.2,-1.00000,0.00000,0.00000,...,0.00009,2.12497,-0.02370,1.09519,0.13223,2285.050,3.550,0.046639,0.407081,0.0165
74,2023-06-21,2025-06-20,730.0,4365.45,7400.0,2451.90,2480.0,-1.00000,0.00000,0.00000,...,0.00003,1.65183,-0.01875,0.81849,0.13399,2465.950,2.600,0.046639,0.434480,0.0165


# merge the daily SSVI parameter on “tmp” dataframe

In [5]:
ssvi_para = pd.read_csv("./ssvi_para.csv")


# 合并DataFrame
ssvi_para.rename(columns={'date': 'QUOTE_DATE'}, inplace=True)

ssvi_para['QUOTE_DATE'] = pd.to_datetime(ssvi_para['QUOTE_DATE'])

ssvi_para = ssvi_para[ssvi_para['QUOTE_DATE'] == '2023-06-21']


df_merged_tmp_ssvi = pd.merge(tmp, ssvi_para, on='QUOTE_DATE', how='left')


df_merged_tmp_ssvi

,QUOTE_DATE,EXPIRE_DATE,DTE,UNDERLYING_LAST,STRIKE,P_BID,P_ASK,P_DELTA,P_GAMMA,P_VEGA,...,len,V,V_prime,theta,rho,eta,gamma,theta_t_loss,w_t_loss,w_t_error
0,2023-06-21,2025-06-20,730.0,4365.45,800.0,2.85,3.7,-0.00231,0.00000,0.51766,...,6426,0.008406,0.0242,0.063574,-0.516952,1.35101,0.5,0.000133,0.013372,0.013372
1,2023-06-21,2025-06-20,730.0,4365.45,1000.0,4.60,5.0,-0.00377,0.00000,0.76942,...,6426,0.008406,0.0242,0.063574,-0.516952,1.35101,0.5,0.000133,0.013372,0.013372
2,2023-06-21,2025-06-20,730.0,4365.45,1200.0,6.10,6.9,-0.00599,0.00004,1.06254,...,6426,0.008406,0.0242,0.063574,-0.516952,1.35101,0.5,0.000133,0.013372,0.013372
3,2023-06-21,2025-06-20,730.0,4365.45,1400.0,8.00,9.0,-0.00821,0.00005,1.41332,...,6426,0.008406,0.0242,0.063574,-0.516952,1.35101,0.5,0.000133,0.013372,0.013372
4,2023-06-21,2025-06-20,730.0,4365.45,1600.0,10.60,11.5,-0.01145,-0.00004,1.84888,...,6426,0.008406,0.0242,0.063574,-0.516952,1.35101,0.5,0.000133,0.013372,0.013372
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,2023-06-21,2025-06-20,730.0,4365.45,6800.0,1910.50,1939.1,-1.00000,0.00000,0.00000,...,6426,0.008406,0.0242,0.063574,-0.516952,1.35101,0.5,0.000133,0.013372,0.013372
72,2023-06-21,2025-06-20,730.0,4365.45,7000.0,2093.60,2118.8,-1.00000,0.00000,0.00000,...,6426,0.008406,0.0242,0.063574,-0.516952,1.35101,0.5,0.000133,0.013372,0.013372
73,2023-06-21,2025-06-20,730.0,4365.45,7200.0,2270.90,2299.2,-1.00000,0.00000,0.00000,...,6426,0.008406,0.0242,0.063574,-0.516952,1.35101,0.5,0.000133,0.013372,0.013372
74,2023-06-21,2025-06-20,730.0,4365.45,7400.0,2451.90,2480.0,-1.00000,0.00000,0.00000,...,6426,0.008406,0.0242,0.063574,-0.516952,1.35101,0.5,0.000133,0.013372,0.013372


# calculate implied volatility based on SSVI

In [6]:
def compute_theta_t(params, t, kappa1, kappa2):
    V, V_prime, theta = params
    theta_t = theta * t + (V - theta) * (1 - np.exp(-kappa1 * t)) / kappa1 + \
              (V_prime - theta) * kappa1 * ((1 - np.exp(-kappa2 * t)) / kappa2 - \
              (1 - np.exp(-kappa1 * t)) / kappa1) / (kappa1 - kappa2)
    return theta_t

def compute_phi(params_w_t, params_theta_t, t, phi_type, kappa1, kappa2):
    rho, eta, gamma = params_w_t
    theta_t = compute_theta_t(params_theta_t, t, kappa1, kappa2)
    if phi_type == "POWER-LAW1":
        return eta / theta_t ** gamma
    elif phi_type == "POWER-LAW2":
        return eta / (theta_t ** gamma * (1 + theta_t) ** (1 - gamma))
    elif phi_type == "SQRT":
        return eta / np.sqrt(theta_t)
    elif phi_type == "HESTON":
        return (1 - (1 - np.exp(-gamma * theta_t)) / (gamma * theta_t)) / (gamma * theta_t)
    else:
        raise ValueError("Invalid phi_type")

def compute_ssvi_inference(params_w_t, params_theta_t, t, k, phi_type, kappa1, kappa2):
    rho, eta, gamma = params_w_t
    theta_t = compute_theta_t(params_theta_t, t, kappa1, kappa2)
    phi = compute_phi(params_w_t, params_theta_t, t, phi_type, kappa1, kappa2)
    return 0.5 * theta_t * (1 + rho * phi * k + np.sqrt((phi * k + rho) ** 2 + 1 - rho ** 2))


# 定义 kappa1 和 kappa2
kappa1 = 5.5
kappa2 = 0.1

# 使用 apply 方法，传递所有必要参数
df_merged_tmp_ssvi['ssvi_iv'] = df_merged_tmp_ssvi.apply(
    lambda row: np.sqrt(
        compute_ssvi_inference(
            params_w_t=[row['rho'], row['eta'], row['gamma']],
            params_theta_t=[row['V'], row['V_prime'], row['theta']],
            t=row['DTE'] / 365,
            k=row['Log-Strike'],
            phi_type="POWER-LAW1",
            kappa1=kappa1,
            kappa2=kappa2
        ) / (row['DTE'] / 365)
    ), axis=1
)

df_merged_tmp_ssvi

,QUOTE_DATE,EXPIRE_DATE,DTE,UNDERLYING_LAST,STRIKE,P_BID,P_ASK,P_DELTA,P_GAMMA,P_VEGA,...,V,V_prime,theta,rho,eta,gamma,theta_t_loss,w_t_loss,w_t_error,ssvi_iv
0,2023-06-21,2025-06-20,730.0,4365.45,800.0,2.85,3.7,-0.00231,0.00000,0.51766,...,0.008406,0.0242,0.063574,-0.516952,1.35101,0.5,0.000133,0.013372,0.013372,0.478133
1,2023-06-21,2025-06-20,730.0,4365.45,1000.0,4.60,5.0,-0.00377,0.00000,0.76942,...,0.008406,0.0242,0.063574,-0.516952,1.35101,0.5,0.000133,0.013372,0.013372,0.450188
2,2023-06-21,2025-06-20,730.0,4365.45,1200.0,6.10,6.9,-0.00599,0.00004,1.06254,...,0.008406,0.0242,0.063574,-0.516952,1.35101,0.5,0.000133,0.013372,0.013372,0.426011
3,2023-06-21,2025-06-20,730.0,4365.45,1400.0,8.00,9.0,-0.00821,0.00005,1.41332,...,0.008406,0.0242,0.063574,-0.516952,1.35101,0.5,0.000133,0.013372,0.013372,0.404460
4,2023-06-21,2025-06-20,730.0,4365.45,1600.0,10.60,11.5,-0.01145,-0.00004,1.84888,...,0.008406,0.0242,0.063574,-0.516952,1.35101,0.5,0.000133,0.013372,0.013372,0.384834
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,2023-06-21,2025-06-20,730.0,4365.45,6800.0,1910.50,1939.1,-1.00000,0.00000,0.00000,...,0.008406,0.0242,0.063574,-0.516952,1.35101,0.5,0.000133,0.013372,0.013372,0.148532
72,2023-06-21,2025-06-20,730.0,4365.45,7000.0,2093.60,2118.8,-1.00000,0.00000,0.00000,...,0.008406,0.0242,0.063574,-0.516952,1.35101,0.5,0.000133,0.013372,0.013372,0.151281
73,2023-06-21,2025-06-20,730.0,4365.45,7200.0,2270.90,2299.2,-1.00000,0.00000,0.00000,...,0.008406,0.0242,0.063574,-0.516952,1.35101,0.5,0.000133,0.013372,0.013372,0.154028
74,2023-06-21,2025-06-20,730.0,4365.45,7400.0,2451.90,2480.0,-1.00000,0.00000,0.00000,...,0.008406,0.0242,0.063574,-0.516952,1.35101,0.5,0.000133,0.013372,0.013372,0.156744


In [10]:
    
import numpy as np

def binomial_tree_american_option_dividend(S0, K, T, r, sigma, N, q, option_type="call"):
    """
    Prices an American option using the binomial tree model, accounting for dividends.
    
    Parameters:
    S0 : float - Initial stock price
    K : float - Strike price
    T : float - Time to maturity in years
    r : float - Risk-free interest rate
    sigma : float - Volatility
    N : int - Number of steps in the binomial tree
    q : float - Continuous dividend yield
    option_type : str - Type of the option ("call" or "put")
    
    Returns:
    float - Estimated option price
    """
    
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    # Adjust the probability to account for dividends
    p = (np.exp((r - q) * dt) - d) / (u - d)

    # Initialize the stock price tree
    stock_prices = np.asarray([S0 * u**j * d**(N-j) for j in range(N+1)])
    
    # Initialize the terminal nodes of the option value tree
    if option_type == "call":
        option_values = np.maximum(stock_prices - K, 0)
    else:  # put
        option_values = np.maximum(K - stock_prices, 0)

    # Backward iterate the option value tree
    for i in range(N-1, -1, -1):
        stock_prices = S0 * u**np.arange(i+1) * d**(i-np.arange(i+1))
        option_values_early = np.maximum(stock_prices - K, 0) if option_type == "call" else np.maximum(K - stock_prices, 0)
        option_values = np.exp(-r * dt) * (p * option_values[1:] + (1 - p) * option_values[:-1])
        option_values = np.maximum(option_values, option_values_early)  # Check for early exercise

    return option_values[0]




# 设置二叉树的步数
N = 100  # 或者根据需要调整

# 定义计算期权价格的函数
def calc_option_price(row, option_type="call"):
    return binomial_tree_american_option_dividend(
        S0=row['UNDERLYING_LAST'], 
        K=row['STRIKE'], 
        T=row['DTE'] / 365,  # 假设DTE是以天为单位，转换为年
        r=row['rf'], 
        sigma=row['ssvi_iv'], 
        N=N,
        q=row['div'],
        option_type=option_type
    )

# 计算每一行的美式看涨期权和看跌期权价格
df_merged_tmp_ssvi['American_Call_Price'] = df_merged_tmp_ssvi.apply(calc_option_price, option_type="call", axis=1)
df_merged_tmp_ssvi['American_Put_Price'] = df_merged_tmp_ssvi.apply(calc_option_price, option_type="put", axis=1)

# 从 df_merged_tmp_ssvi DataFrame 中提取 'American_Call_Price' 和 'American_Put_Price' 列
df_American_prices = df_merged_tmp_ssvi[['American_Call_Price', 'American_Put_Price']]

df_American_prices

,American_Call_Price,American_Put_Price
0,3565.450000,1.644113
1,3365.702867,3.165030
2,3170.726112,5.399111
3,2980.606035,8.378563
4,2794.794949,12.639003
...,...,...
71,14.214835,2434.550000
72,10.940913,2634.550000
73,8.984673,2834.550000
74,7.095261,3034.550000


In [11]:
import numpy as np

def binomial_tree_european_option_vectorized(S0, K, T, r, sigma, N, q, option_type="call"):
    """
    Prices a European option using the binomial tree model, accounting for dividends.

    Parameters:
    S0 : float - Initial stock price
    K : float - Strike price
    T : float - Time to maturity in years
    r : float - Risk-free interest rate
    sigma : float - Volatility
    N : int - Number of steps in the binomial tree
    q : float - Continuous dividend yield
    option_type : str - Type of the option ("call" or "put")

    Returns:
    float - Estimated option price
    """
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    # Adjust the calculation of p to account for dividends
    p = (np.exp((r - q) * dt) - d) / (u - d)

    # Initialize stock price tree - vectorized operation
    stock_prices = S0 * u**np.arange(N + 1) * d**(N - np.arange(N + 1))

    # Initialize the terminal nodes of the option value tree - vectorized operation
    if option_type == "call":
        option_values = np.maximum(stock_prices - K, 0)
    else:  # put
        option_values = np.maximum(K - stock_prices, 0)

    # Backward iterate the option value tree - vectorized operation
    for i in range(N-1, -1, -1):
        option_values = np.exp(-r * dt) * (p * option_values[1:] + (1 - p) * option_values[:-1])

    return option_values[0]


# 设置二叉树的步数
N = 100  # 或者根据需要调整

# 定义计算期权价格的函数
def calc_option_price(row, option_type="call"):
    return binomial_tree_european_option_vectorized(
        S0=row['UNDERLYING_LAST'], 
        K=row['STRIKE'], 
        T=row['DTE'] / 365,  # 假设DTE是以天为单位，转换为年
        r=row['rf'], 
        sigma=row['ssvi_iv'], 
        N=N,
        q=row['div'],
        option_type=option_type
    )

# 计算每一行的美式看涨期权和看跌期权价格
df_merged_tmp_ssvi['European_Call_Price'] = df_merged_tmp_ssvi.apply(calc_option_price, option_type="call", axis=1)
df_merged_tmp_ssvi['European_Put_Price'] = df_merged_tmp_ssvi.apply(calc_option_price, option_type="put", axis=1)

# 从 df_merged_tmp_ssvi DataFrame 中提取 'American_Call_Price' 和 'American_Put_Price' 列
df_European_prices = df_merged_tmp_ssvi[['European_Call_Price', 'European_Put_Price']]

df_European_prices

,European_Call_Price,European_Put_Price
0,3496.613556,1.624263
1,3315.922889,3.121574
2,3135.925861,5.312524
3,2956.641248,8.215889
4,2778.646323,12.408942
...,...,...
71,14.214835,1984.864882
72,10.940913,2163.778937
73,8.984673,2344.010676
74,7.095261,2524.309242


# Calculating EEP

In [13]:
result_df = pd.DataFrame()

result_df['EEP_call'] = df_American_prices['American_Call_Price'] - df_European_prices['European_Call_Price'].round(4)

result_df['EEP_Put'] = df_American_prices['American_Put_Price'] - df_European_prices['European_Put_Price'].round(4)

result_df

,EEP_call,EEP_Put
0,68.836400,0.019813
1,49.779967,0.043430
2,34.800212,0.086611
3,23.964835,0.162663
4,16.148649,0.230103
...,...,...
71,0.000035,449.685100
72,0.000013,470.771100
73,-0.000027,490.539300
74,-0.000039,510.240800
